In [38]:
import pandas as pd
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import spacy
import pandas as pd
import os
import shutil
import pickle 
import spacy
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np
import spacy
import  pickle
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
from gensim import corpora, models, similarities, downloader
from tqdm import tqdm_notebook
import gensim
from gensim.models import Word2Vec
import re


C:\Users\pc\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
df = pd.read_pickle('discussions_corrected.p')

In [36]:
df.head(10)

,title,type,year,post
0,Better Call Saul,linear,2017,Walter. And there the chain ends.
1,Better Call Saul,linear,2016,I love this show. But it's hard to argue again...
2,Better Call Saul,linear,2017,What am I missing? A lot of reference to ribs...
3,Better Call Saul,linear,2018,"Oh come on Mike, he's a good little boy."
4,Better Call Saul,linear,2017,Look again 👀
5,Better Call Saul,linear,2018,"I'm watching with my mum and she was like ""Kim..."
6,Better Call Saul,linear,2020,"Maybe when Jimmy gets the $7 million to ABQ, t..."
7,Better Call Saul,linear,2016,Didn't VG say no other BB characters would be ...
8,Better Call Saul,linear,2017,I remember spooning (gahhk that word is so bad...
9,Better Call Saul,linear,2018,How did Gus find out mike knew about Nacho?


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   50000 non-null  object
 1   type    50000 non-null  object
 2   year    50000 non-null  int64 
 3   post    50000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


In [30]:
types = df['type'].unique().tolist() # linear, netflix, extremely inefficient line
years = df['year'].unique().tolist() # we have data from 2011 to 2020
shows = df['title'].unique().tolist()

In [32]:
types

['linear', 'netflix']

In [33]:
years

[2017, 2016, 2018, 2020, 2012, 2013, 2011, 2014, 2019, 2015]

In [34]:
shows

['Better Call Saul',
 'Black Mirror',
 'Breaking Bad',
 'Dark',
 'Fargo',
 'Game of Thrones',
 'House of Cards',
 'La Casa de Papel',
 'Mindhunter',
 'Mr. Robot',
 'Orange is the New Black',
 'Ozark',
 'Stranger Things',
 'Succession',
 'The Crown',
 'The Mandelorian',
 'The Newsroom',
 'The Witcher',
 'True Detective',
 'Twin Peaks']

In [35]:
df['type'].value_counts().tolist()

[35269, 14731]

In [12]:
df['title'].value_counts()

Game of Thrones            15462
Breaking Bad                6424
Better Call Saul            5268
Black Mirror                4720
Stranger Things             2891
True Detective              2721
Twin Peaks                  2399
Dark                        2004
Ozark                       1417
Mr. Robot                   1347
Orange is the New Black     1208
The Witcher                 1188
Fargo                        680
Mindhunter                   657
The Newsroom                 484
Succession                   389
The Crown                    338
House of Cards               158
La Casa de Papel             150
The Mandelorian               95
Name: title, dtype: int64

In [19]:
df['year'].value_counts()

2017    12830
2016     7734
2015     7118
2019     6608
2018     4332
2020     3851
2012     3342
2013     2758
2014      839
2011      588
Name: year, dtype: int64

In [40]:
df_linear = df[df.type == 'linear']
df_netflix = df[df.type == 'netflix']

In [41]:
nlp = spacy.load("en_core_web_sm")

In [42]:
processed_texts = [text for text in tqdm_notebook(nlp.pipe(df_linear['post'], 
                                              n_process=-1, # maximum number of threads
                                              disable=["ner",
                                                       "parser"]),
                                         total=len(df_linear['post']))]


C:\Users\pc\anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [43]:
tokenized_df_linear = [[token.lemma_ for token in text
               if not token.is_punct and not token.is_stop]
              for text in processed_texts]      

In [45]:
tokenized_df_linear = [[re.sub(r'\W+', '', word) for word in text] for text in tokenized_df_linear] #we remove the words we dont want
tokenized_df_linear[:1]

[['Walter', 'chain', 'end']]

In [47]:
tokenized_df_linear2 = [[re.sub(r'\b\w{1,3}\b', '', word) for word in text] for text in tokenized_df_linear] # we remove the words that are 3 or more,in my opinion they are noise
tokenized_df_linear2[:1]

[['Walter', 'chain', '']]

In [48]:
processed_texts = [text for text in tqdm_notebook(nlp.pipe(df_netflix['post'], 
                                              n_process=-1, # maximum number of threads
                                              disable=["ner",
                                                       "parser"]),
                                         total=len(df_netflix['post']))]


tokenized_df_netflix = [[token.lemma_ for token in text
               if not token.is_punct and not token.is_stop]
              for text in processed_texts]                                        


C:\Users\pc\anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [49]:
tokenized_df_netflix = [[re.sub(r'\W+', '', word) for word in text] for text in tokenized_df_netflix] 
tokenized_df_netflix[:1]

[['think', 'rabbit', 'ending', 'real', 'ending', 'traincrasch']]

In [50]:
tokenized_df_netflix2 = [[re.sub(r'\b\w{1,3}\b', '', word) for word in text] for text in tokenized_df_netflix] 
tokenized_df_netflix2[:2]

[['think', 'rabbit', 'ending', 'real', 'ending', 'traincrasch'],
 ['wipe', '', 'tracker', 'realize', 'fate']]

In [52]:
#first we create the dictionary from the netflix
from gensim.corpora import Dictionary

MIN_DF = 5 # minium document frequency
MAX_DF = 0.6 # maximum document frequency


dictionary = Dictionary(tokenized_df_netflix2) # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF, 
                           no_above=MAX_DF)
corpus_netflix = [dictionary.doc2bow(text) for text in tokenized_df_netflix2]

In [54]:
#second we create the dictionary from the linear
from gensim.corpora import Dictionary

MIN_DF = 5 # minium document frequency
MAX_DF = 0.6 # maximum document frequency


dictionary = Dictionary(tokenized_df_linear2) # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF, 
                           no_above=MAX_DF)
corpus_linear = [dictionary.doc2bow(text) for text in tokenized_df_linear2]

In [37]:
from gensim.models.wrappers import LdaMallet

First of all i need to persuade the television producer so i have to make an operationalization that is going to be understandable.I need to measure the engagment of the audience ether to linear or Netflix release